In [20]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [21]:
# 查看当前kernerl下的package
# !pip list --format=columns

In [22]:
# 显示cell运行时长
%load_ext klab-autotime

In [23]:
import numpy as np
import pandas as pd
import time
import math
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn import metrics
from sklearn import utils
from sklearn.model_selection import KFold
from fastText import train_supervised
import lightgbm as lgb
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument, LabeledSentence
from tqdm import tqdm
import Levenshtein
import gc

In [24]:
path="/home/kesci/input/bytedance/first-round/"
train = pd.read_csv(path+'train.csv', header=None, chunksize=10000000)
test = pd.read_csv(path+'test.csv', header=None)
pre_df = test[[0, 2]]

In [25]:
def q_auc(df):
    invalid_count = 0
    query_ids = df['query_id'].unique()
    aucs = []
    for query_id in query_ids:
        current = df[df['query_id'] == query_id]
        if sum(current['label']) not in [0, len(current['label'])]:
            auc = roc_auc_score(current['label'], current['prediction'])  # 这里计算qauc
        else:
            auc = 0.5
            invalid_count += 1
        aucs.append(auc)
    print(f'q_auc invalid_count = {invalid_count}')
    return np.mean(aucs)

In [26]:
# ct = CountVectorizer()
# d = train[1] + ' ' +train[3]
# x_train = ct.fit_transform(d)

# d = test[1] + ' ' +test[3]
# x_test = ct.transform(d)

# del d

In [27]:

# #lr+CountVectorizer

# # def getfTitleOhot(df):
# #     a = np.zeros((df.shape[0], 20), dtype=np.int8)
# #     for i in range(len(d)):
# #         b = d[i].split(' ')
# #         for j in b:
# #             a[i][int(j)] = int(j)
            
# #     return a

# kf_n = 5
# kf = KFold(n_splits=kf_n, shuffle=True, random_state=2)
# pre = 0

# # model = SGDClassifier(penalty='l1', random_state = 2, shuffle = True, loss='log', alpha=0.01)
# model = LogisticRegression(penalty='l1', C=0.14, n_jobs=-1)
# x = 1
# for df in train:
    
#     ct = CountVectorizer(ngram_range=(1, 3), min_df=50)
#     # ct = CountVectorizer(ngram_range=(1, 2))
#     # ct = TfidfVectorizer(sublinear_tf=True,ngram_range=(1, 2), max_df=0.5, analyzer='word', norm='l1')
    
#     print('第%d次训练开始'%x)
    
#     #训练集
#     d = df[1] + ' ' +df[3]
#     y_train = df[4].copy()
#     x_train = ct.fit_transform(d)
    
#     # title_oneh = np.array(pd.get_dummies(df[2]))
#     # x_train = np.hstack((x_train, title_oneh))
#     # del title_oneh
        
#     print('训练集处理完成')

#     #测试集
#     d = test[1] + ' ' +test[3]
#     x_test = ct.transform(d)
    
#     # title_oneh = np.array(pd.get_dummies(test[2]))
#     # x_train = np.hstack((x_train, title_oneh))
#     # del title_oneh
    
#     print('测试集处理完成')
    
#     del d

        
#     for x_tr, x_vail in kf.split(x_train):
#         x_t1 = x_train[x_tr]
#         y_t1 = y_train.iloc[x_tr]
        
#         x_t2 = x_train[x_vail]
#         y_t2 = y_train.iloc[x_vail]
        
#         model.fit(x_t1, y_t1)
#         print(metrics.roc_auc_score(y_t2, model.predict_proba(x_t2)[:, 1]))
#         pre += model.predict_proba(x_test)[:, 1]
        
#     # model.fit(x_train, y_train)
#     # pre += model.predict_proba(x_test)[:, 1]
        
#     del x_train
#     del x_test
#     del ct
    
#     print('第%d次训练结束'%x)

#     x += 1
        

    
# pre1 = pre/(kf_n*(x-1))
# # pre1 = pre/(x-1)

# pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df['prediction'] = pre1
# pre_df.to_csv('lr_ct_500wEoch_无cv.csv', index=False, header=False)

# '''
# 5折交叉验证+500w一次,验证得分0.574
# 2折交叉验证+1000w一次,验证得分0.568
# '''

In [28]:
def lcsubstr_lens(s1, s2): 
    m=[[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]  #生成0矩阵，为方便后续计算，比字符串长度多了一列
    mmax=0   #最长匹配的长度
    p=0  #最长匹配对应在s1中的最后一位
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i]==s2[j]:
                m[i+1][j+1]=m[i][j]+1
                if m[i+1][j+1]>mmax:
                    mmax=m[i+1][j+1]
                    p=i+1
    return mmax


#
def lcseque_lens(s1, s2): 
     # 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
    m = [ [ 0 for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    # d用来记录转移方向
    d = [ [ None for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 

    for p1 in range(len(s1)): 
        for p2 in range(len(s2)): 
            if s1[p1] == s2[p2]:            #字符匹配成功，则该位置的值为左上方的值加1
                m[p1+1][p2+1] = m[p1][p2]+1
                d[p1+1][p2+1] = 'ok'          
            elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向
                m[p1+1][p2+1] = m[p1+1][p2] 
                d[p1+1][p2+1] = 'left'          
            else:                           #上值大于左值，则该位置的值为上值，并标记方向up
                m[p1+1][p2+1] = m[p1][p2+1]   
                d[p1+1][p2+1] = 'up'         
    (p1, p2) = (len(s1), len(s2)) 
    s = [] 
    while m[p1][p2]:    #不为None时
        c = d[p1][p2]
        if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个
            s.append(s1[p1-1])
            p1 -= 1
            p2 -= 1 
        if c == 'left':  #根据标记，向左找下一个
            p2 -= 1
        if c == 'up':   #根据标记，向上找下一个
            p1 -= 1
    return len(s)
    
def compute_cosine(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算余弦相似度
    sum = 0
    sq1 = 0
    sq2 = 0
    for i in range(len(vect1)):
        sum += vect1[i] * vect2[i]
        sq1 += pow(vect1[i], 2)
        sq2 += pow(vect2[i], 2)
    try:
        result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
    except ZeroDivisionError:
        result = 0.0
    return result

def Pehrson(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算Pehrson
    x = np.vstack([vect1, vect2])
    return np.corrcoef(x)[0][1]


In [29]:
def getfeature(df):
    # df.drop_duplicates(inplace=True)
    since = time.time()
    
    df['query_len'] = df.apply(lambda x: len(x[1].split(' ')), axis=1)
    df['query_len'] = df['query_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def get_query_unique(x):
        a = len(set(x[1].split(' '))-set(x[3].split(' ')))
        return a
    df['query_unique'] = df.apply(get_query_unique, axis=1)
    df['query_unique'] = df['query_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#####################################3
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['title_len'] = df.apply(lambda x: len(x[3].split(' ')), axis=1)
    df['title_len'] = df['title_len'].astype(np.int32)
    
    time_elapsed = time.time() - since######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def get_query_unique(x):
        a = len(set(x[3].split(' '))-set(x[1].split(' ')))
        return a
    
    df['title_unique'] = df.apply(get_query_unique, axis=1)
    df['title_unique'] = df['title_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['dif'] = (df['query_len'] - df['title_len']).abs()
    df['dif'] = df['dif'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def query_title_same_len(x):
        a = len(x[3].split(' ') and x[1].split(' '))
        return a
    df['query_title_same_len'] = df.apply(query_title_same_len, axis=1)
    df['query_title_same_len'] = df['query_title_same_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    def samelen_query_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[1].split(' '))
    df['samelen_query_rate'] = df.apply(samelen_query_rate, axis=1)
    df['samelen_query_rate'] = df['samelen_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def samelen_title_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[3].split(' '))
    df['samelen_title_rate'] = df.apply(samelen_title_rate, axis=1)
    df['samelen_title_rate'] = df['samelen_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    
    def q_t_all_len(x):
        a = len(x[1].split(' ')) + len(x[3].split(' '))
        return a
    df['q_t_all_len'] = df.apply(q_t_all_len, axis=1)
    df['q_t_all_len'] = df['q_t_all_len'].astype(np.int32)
    
    time_elapsed = time.time() - since##########################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_all_word_len(x):
        a = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return a
    df['q_t_all_word_len'] = df.apply(q_t_all_word_len, axis=1)
    df['q_t_all_word_len'] = df['q_t_all_word_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['q_t_diff_len'] = df.apply(lambda x: len(set(x[1].split(' '))^set(x[3].split(' '))), axis=1)
    df['q_t_diff_len'] = df['q_t_diff_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_diff_q_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[1].split(' ')))
    df['q_t_diff_q_rate'] = df.apply(q_t_diff_q_rate, axis=1)
    df['q_t_diff_q_rate'] = df['q_t_diff_q_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_diff_t_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[3].split(' ')))
    df['q_t_diff_t_rate'] = df.apply(q_t_diff_t_rate, axis=1)
    df['q_t_diff_t_rate'] = df['q_t_diff_t_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    
    def query_diff_title(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['query_diff_title'] = df.apply(query_diff_title, axis=1)
    df['query_diff_title'] = df['query_diff_title'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def query_diff_title_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['query_diff_title_rate'] = df.apply(query_diff_title_rate, axis=1)
    df['query_diff_title_rate'] = df['query_diff_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def title_diff_query(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['title_diff_query'] = df.apply(title_diff_query, axis=1)
    df['title_diff_query'] = df['title_diff_query'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def title_diff_query_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['title_diff_query_rate'] = df.apply(title_diff_query_rate, axis=1)
    df['title_diff_query_rate'] = df['title_diff_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def dice(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' '))) + len(set(x[3].split(' ')))
        return np.float32(2*a1/a2)
    df['dice'] = df.apply(dice, axis=1)
    df['dice'] = df['dice'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def jaccord(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return np.float32(a1/a2)
    df['jaccord'] = df.apply(jaccord, axis=1)
    df['jaccord'] = df['jaccord'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['query_count'] = df.groupby(1)[1].transform('count')
    df['query_count'] = df['query_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['title_count'] = df.groupby(3)[3].transform('count')
    df['title_count'] = df['title_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['lcseque_lens'] = df.apply(lambda x: lcseque_lens(x[1], x[3]), axis=1)
    df['lcseque_lens'] = df['lcseque_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['lcsubstr_lens'] = df.apply(lambda x: lcsubstr_lens(x[1], x[3]), axis=1)
    df['lcsubstr_lens'] = df['lcsubstr_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    

    df['ratio'] = df.apply(lambda x: Levenshtein.ratio(x[1], x[3]), axis=1)
    df['ratio'] = df['ratio'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    

    df['jaro'] = df.apply(lambda x: Levenshtein.jaro(x[1], x[3]), axis=1)
    df['jaro'] = df['jaro'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['jaro_winkler'] = df.apply(lambda x: Levenshtein.jaro_winkler(x[1], x[3]), axis=1)
    df['jaro_winkler'] = df['jaro_winkler'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    df['cosine_similarity'] = df.apply(lambda x: compute_cosine(x[1], x[3]), axis=1)
    df['cosine_similarity'] = df['cosine_similarity'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    df['levenshtein'] = df.apply(lambda x: Levenshtein.distance(x[1], x[3]), axis=1)
    df['levenshtein'] = df['levenshtein'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    
    df['Pehrson'] = df.apply(lambda x:Pehrson(x[1], x[3]), axis=1)
    df['Pehrson'] = df['Pehrson'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['list_dis'] = df.apply(lambda x: Levenshtein.seqratio(x[1].split(' '), x[3].split(' ')), axis=1)
    df['list_dis'] = df['list_dis'].astype(np.float32)
    
    time_elapsed = time.time() - since
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    

    try:
        df.drop([0, 1, 2, 3, 4], axis=1, inplace=True)
    except:
        df.drop([0, 1, 2, 3], axis=1, inplace=True)
        
    return df


In [30]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
#        mask = mask.T[0]
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
#        mask = mask.T[0]
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
#    ys = ys.reshape(ys.shape[0], 1)
    yn = np.zeros(xn.shape[0])
#    yn = yn.reshape(yn.shape[0], 1)
    print(xs.shape)
    print(xn.shape)
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y


In [31]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

# #测试集
# print('测试集')
# # d = pd.DataFrame(test[1] + ' ' +test[3])
# # del test
# test.drop([0, 2], axis=1, inplace=True)
# test['label'] = 0
# test['cut_review'] = test[1].apply(lambda x: [w for w in x.split(' ') if w != ' '])
# test_query_tagged = test.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r['label']]), axis=1)
# test['cut_review'] = test[3].apply(lambda x: [w for w in x.split(' ') if w != ' '])
# test_title_tagged = test.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r['label']]), axis=1)
# del test
# print('测试集分词完成')

# kf_n = 5
# kf = KFold(n_splits=kf_n, shuffle=True, random_state=2)

# pre_lr = 0
# pre_lgb = 0

# lr = LogisticRegression(penalty='l1', C=0.14, n_jobs=4)

x = 1
for df in train:
    if x != 10:
        print(x)
        x += 1
        continue
    
    print('第%d次处理开始'%x)
    
    test[4] = 0
    print('合并训练集和测试集')
    len_train = len(df)
    df = pd.concat((df, test))
    del test


    #训练集
    # n_train = np.random.choice(range(len(df)), 5000000, replace=False)
    # df_train = df.iloc[n_train, :]
#    df['all_review'] = pd.DataFrame(df[1] + ' ' +df[3])
    # d = pd.DataFrame(df_train[1] + ' ' +df_train[3])
    # d['label'] = df_train[4]
    df['cut_review'] = df[1].apply(lambda x: [w for w in x.split(' ') if w != ' '])
    train_query_tagged = df.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r[4]]), axis=1)
    df['cut_review'] = df[3].apply(lambda x: [w for w in x.split(' ') if w != ' '])
    train_title_tagged = df.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r[4]]), axis=1)

    del df

    # #验证集
    # n_vail = np.random.choice(range(len(df)), 1000000, replace=False)
    # df_vail = df.iloc[n_vail, :]
    # d = pd.DataFrame(df_vail[1] + ' ' +df_vail[3])
    # d['label'] = df_vail[4]
    # d['cut_review'] = d[0].apply(lambda x: [w for w in list(jieba.cut(x)) if w != ' '])
    # vail_tagged = d.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r['label']]), axis=1)
    
    # del d
    # del n_vail
    # del df_vail
    # del df
    

    # #测试集
    # d = pd.DataFrame(test[1] + ' ' +test[3])
    # d['label'] = 0
    # d['cut_review'] = d[0].apply(lambda x: [w for w in list(jieba.cut(x)) if w != ' '])
    # test_tagged = d.apply(lambda r: TaggedDocument(words=r['cut_review'], tags=[r['label']]), axis=1)

    
    model_dbow = Doc2Vec(dm=0,  negative=5, hs=0, min_count=100, sample = 0, workers=2)
    model_dbow.build_vocab([x for x in tqdm(train_query_tagged.values)])

    for epoch in range(5):
        model_dbow.train(utils.shuffle([x for x in tqdm(train_query_tagged.values)]), 
                                total_examples=len(train_query_tagged.values), epochs=2)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
        
    
    print('第%d次处理完成训练开始'%x)
    
    #训练集向量化
    print('query训练')
    y_train, x_train = vec_for_learning(model_dbow, train_query_tagged)
    del train_query_tagged
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    # x_train, y_train = augment(x_train, y_train)
    # x_train = np.hstack((x_train, df_train.iloc[:lentrain, :]))
    print(x_train[:len_train].shape, x_train[len_train:].shape)
    np.save('train_query_doc2vec_last1000W.npy', x_train[:len_train])
    np.save('train_label_doc2vec_last1000W.npy', y_train[:len_train])
    np.save('test_query_doc2vec_last1000W.npy', x_train[len_train:])
    del x_train
    del y_train
    

    # #验证集向量化
    # y_vail, x_vail = vec_for_learning(model_dbow, vail_tagged)
    # del vail_tagged
    # x_vail = np.array(x_vail)
    # y_vail = np.array(y_vail)
    # print(x_vail.shape)
    
    #测试集向量化
    # _, x_test = vec_for_learning(model_dbow, test_query_tagged)
    # del test_query_tagged
    # x_test = np.array(x_test)
    # # x_test = np.hstack((x_test, df_train.iloc[lentrain:, :]))
    # print(x_test.shape)
    # np.save('test_query_doc2vec_last1000W.npy', x_test)
    # del x_test
    
    
    model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    del model_dbow

    print('title训练')
    model_dbow = Doc2Vec(dm=0,  negative=5, hs=0, min_count=100, sample = 0, workers=2)
    model_dbow.build_vocab([x for x in tqdm(train_title_tagged.values)])

    for epoch in range(5):
        model_dbow.train(utils.shuffle([x for x in tqdm(train_title_tagged.values)]), 
                                total_examples=len(train_title_tagged.values), epochs=2)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
        
    
    y_train, x_train = vec_for_learning(model_dbow, train_title_tagged)
    del train_title_tagged
    x_train = np.array(x_train)
    # x_train, y_train = augment(x_train, y_train)
    # x_train = np.hstack((x_train, df_train.iloc[:lentrain, :]))
    print(x_train.shape)
    np.save('train_title_doc2vec_last1000W.npy', x_train[:len_train])
    np.save('test_title_doc2vec_last1000W.npy', x_train[len_train:])
    del x_train
    del y_train
    
    # #测试集向量化
    # _, x_test = vec_for_learning(model_dbow, test_title_tagged)
    # del test_title_tagged
    # x_test = np.array(x_test)
    # # x_test = np.hstack((x_test, df_train.iloc[lentrain:, :]))
    # print(x_test.shape)
    # np.save('test_title_doc2vec_last1000W.npy', x_test)
    # del x_test
    
    print('结束')
    break
    
    
    

In [32]:
# kf_n = 5
# kf = KFold(n_splits=kf_n, shuffle=True)

# pre_lr = 0

# lr = LogisticRegression(penalty='l2', C=0.3, n_jobs=4)

# for x_tr, x_vail in kf.split(x_train):
        
#         lr.fit(x_train[x_tr], y_train[x_tr])
#         print(metrics.roc_auc_score(y_train[x_vail], lr.predict_proba(x_train[x_vail])[:, 1]))
#         print(metrics.log_loss(y_train[x_vail], lr.predict_proba(x_train[x_vail])[:, 1]))
#         pre_lr += lr.predict_proba(x_test)[:, 1]

# pre_lr /= kf_n

# pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df['prediction'] = pre_lr
# pre_df.to_csv('lr_doc2vec_last500w.csv', index=False, header=False)

In [33]:
# pre_lgb = 0

# random_state = 2

# # lgb_params = {
# #     "objective" : "binary",
# #     # "metric" : "auc",
# #     "boosting": 'gbdt',
# #     "max_depth" : -1,
# #     "num_leaves" : 5,
# #     "learning_rate" : 0.1,
# #     "bagging_freq": 5,
# #     "bagging_fraction" : 0.6,
# #     "feature_fraction" : 0.05,
# #     "min_data_in_leaf": 700,
# #     "min_sum_heassian_in_leaf": 10,
# #     "tree_learner": "serial",
# #     "boost_from_average": "false",
# # #    "lambda_l1" : 5,
# # #    "lambda_l2" : 5,
# #     "bagging_seed" : random_state,
# #     "verbosity" : 1,
# #     "seed": random_state
# # }

# lgb_params = {
#     'boosting_type': 'gbdt',
#     'objective': "binary",
#     'metric': ['binary_logloss',"auc"],
#     'num_leaves': 32,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 1
# }


# for x_tr, x_vail in kf.split(x_train):
        
#         #当x_train为ndarray时，下列数据集的划分会进行复制而占用内存
        
#         # x_t1 = x_train[x_tr]
#         # y_t1 = y_train[x_tr]
        
#         # x_t2 = x_train[x_vail]
#         # y_t2 = y_train[x_vail]
            
#         lgbtr1=lgb.Dataset(x_train[x_tr], y_train[x_tr])
#         lgbtr2=lgb.Dataset(x_train[x_vail], y_train[x_vail])
            
#         evals_result = {}
#         gbm = lgb.train(lgb_params,
#                           lgbtr1,
#                           num_boost_round=1000,
#                           valid_sets=lgbtr2,
#                           verbose_eval=50,
#                           early_stopping_rounds=50,
#                           evals_result=evals_result)
                          
#         print(metrics.roc_auc_score(y_train[x_vail], gbm.predict(x_train[x_vail])))
#         pre_lgb += gbm.predict(x_test, num_iteration=gbm.best_iteration)
        
# pre_lgb /= kr_n

# pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df['prediction'] = pre_lgb
# pre_df.to_csv('lgb_doc2vec_last500w.csv', index=False, header=False)


In [34]:
# !./kesci_submit -token 5ebea4ff5f5fa8e0 -file lgb_doc2vec_last500w.csv